In [1]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
library(themis)
library(parsnip)
options(repr.matrix.max.rows = 6)
source("cleanup.R")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [2]:
players <- read_csv("https://raw.githubusercontent.com/Snowy129/DSCI100-Project/refs/heads/main/players.csv")
head(players)

sessions <- read_csv("https://raw.githubusercontent.com/Snowy129/DSCI100-Project/refs/heads/main/sessions.csv")
head(sessions)

Rows: 196 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): experience, hashedEmail, name, gender
dbl (2): played_hours, age
lgl (3): subscribe, individualId, organizationName

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
<chr>,<lgl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
Pro,TRUE,f6daba428a5e19a3d47574858c13550499be23603422e6a0ee9728f8b53e192d,30.3,Morgan,Male,9,NA,NA
Veteran,TRUE,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa939732842f2312358a88e9,3.8,Christian,Male,17,NA,NA
Veteran,FALSE,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3c5a9d2118eb7ccbb28,0.0,Blake,Male,17,NA,NA
Amateur,TRUE,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4fa7a5a659ff443a0eb5,0.7,Flora,Female,21,NA,NA
Regular,TRUE,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb0af4d48fcce2420f3e,0.1,Kylie,Male,21,NA,NA
Amateur,TRUE,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee0430055a44fa92f977,0.0,Adrian,Female,17,NA,NA


Rows: 1535 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): hashedEmail, start_time, end_time
dbl (2): original_start_time, original_end_time

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


hashedEmail,start_time,end_time,original_start_time,original_end_time
<chr>,<chr>,<chr>,<dbl>,<dbl>
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,30/06/2024 18:12,30/06/2024 18:24,1.71977e+12,1.71977e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,17/06/2024 23:33,17/06/2024 23:46,1.71867e+12,1.71867e+12
f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3398304c7ae42581fdc,25/07/2024 17:34,25/07/2024 17:57,1.72193e+12,1.72193e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,25/07/2024 03:22,25/07/2024 03:58,1.72188e+12,1.72188e+12
36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f575d4acc9cf487c4686,25/05/2024 16:01,25/05/2024 16:12,1.71665e+12,1.71665e+12
bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431d8aa0c4bf95ccee6bf,23/06/2024 15:08,23/06/2024 17:10,1.71916e+12,1.71916e+12


In [3]:
clean_players1 <- players |>
    mutate(gender = as_factor(gender), age = as.integer(age), experience = as_factor(experience)) |>
    select(played_hours, gender, age, experience, subscribe) |>
    filter(age >= 17)
head(clean_players1)




played_hours,gender,age,experience,subscribe
<dbl>,<fct>,<int>,<fct>,<lgl>
3.8,Male,17,Veteran,TRUE
0.0,Male,17,Veteran,FALSE
0.7,Female,21,Amateur,TRUE
0.1,Male,21,Regular,TRUE
0.0,Female,17,Amateur,TRUE
0.0,Female,19,Regular,TRUE


## KNN Regression Attempt

In [4]:

#KNN regression Wrangling
clean_players_with_outliers <- players |>
    mutate(age = as.integer(age)) |>
    select(played_hours,age,experience) |>
    arrange(-played_hours)
head(clean_players_with_outliers)

#Creating a data frame to find the outliers for played hours
clean_players_cutoff <- players |>
    select(played_hours) |>
    arrange(-played_hours)

#finding outlier cutoff for played hours
cutoff_high = quantile(clean_players_cutoff, probs = 0.975, na.rm = TRUE)
cutoff_high

#removing age and played_hours outliers
clean_players_knn <- clean_players_with_outliers |>
    filter(age >= 17) |>
    filter(played_hours < cutoff_high) |>
    filter(played_hours != 0) |>
    arrange(-played_hours)
clean_players_knn



played_hours,age,experience
<dbl>,<int>,<chr>
223.1,17,Regular
218.1,20,Regular
178.2,19,Regular
150.0,16,Amateur
56.1,23,Amateur
53.9,17,Amateur


97.5% 
54.175

played_hours,age,experience
<dbl>,<int>,<chr>
53.9,17,Amateur
48.4,17,Amateur
32.0,22,Amateur
⋮,⋮,⋮
0.1,17,Veteran
0.1,18,Regular
0.1,44,Veteran


In [5]:
set.seed(1234)

#Splitting the Data into Testing and Training
players_split <- initial_split(clean_players_knn, prop = 0.80, strata = played_hours)
players_testing <- testing(players_split)
players_training <- training(players_split)



#PLAN#

- Do first knn regression with all predictors
    - make sure to do vfold and cross analysis
    - find the weights of each predictor and find RMSPE

- Do second knn regression with the top 3 highest weights
    - do vfold
    - find the top 2 weights and the RMSPE

- Do final knn regression with only the top 2 weights
    - do vfold
    - find the RMSPE

- Compare all 3 RMSPE and decide which model is the best


In [28]:
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 5) |>
    set_engine("kknn") |>
    set_mode("regression")

knn_recipe <- recipe(played_hours ~ age, data = players_training) |>
    step_scale(age) |>
    step_center(age)

#player_vfold <- vfold_cv(players_training, v = 5, strata = played_hours) 

#grid_vals <- tibble(neighbors = seq(from = 1, to = 200, by = 3))

all_pred_workflow <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_spec) |>
#    tune_grid(resamples = player_vfold, grid = grid_vals) |>
    fit(data = players_training)
  #  collect_metrics() |>
  #  filter(.metric == "rmse")

all_pred_workflow    

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: nearest_neighbor()

── Preprocessor ────────────────────────────────────────────────────────────────
2 Recipe Steps

• step_scale()
• step_center()

── Model ───────────────────────────────────────────────────────────────────────

Call:
kknn::train.kknn(formula = ..y ~ ., data = data, ks = min_rows(5,     data, 5), kernel = ~"rectangular")

Type of response variable: continuous
minimal mean absolute error: 2.717895
Minimal mean squared error: 50.03062
Best kernel: rectangular
Best k: 5

In [29]:
knn_metrics <- all_pred_workflow |>
    collect_metrics(players_training)

ERROR: [1m[33mError[39m in `collect_metrics()`:[22m
[33m![39m No `collect_metric()` exists for this type of object.


## Linear Regression Attempt

In [16]:
#Linear Regression Wrangling
clean_players_with_outliers <- players |>
    mutate(age = as.integer(age), subscribe = as.numeric(subscribe)) |>
    select(played_hours,age, subscribe) |>
    arrange(-played_hours)
head(clean_players_with_outliers)

#Cutting off outliers of played_hours
cutoff_high = quantile(clean_players_with_outliers, probs = 0.975, na.rm = TRUE)
cutoff_high

clean_players <- clean_players_with_outliers |>
    filter(age >= 17) |>
    filter(played_hours < cutoff_high) |>
    filter(played_hours != 0) |>
    arrange(-played_hours)
clean_players


played_hours,age,subscribe
<dbl>,<int>,<dbl>
223.1,17,1
218.1,20,1
178.2,19,1
150.0,16,1
56.1,23,1
53.9,17,1


97.5% 
 39.3

played_hours,age,subscribe
<dbl>,<int>,<dbl>
32.0,22,1
23.7,24,1
18.5,49,1
⋮,⋮,⋮
0.1,17,1
0.1,18,0
0.1,44,1


In [17]:
knn_spec <- linear_reg() |>
    set_engine("lm") |>
    set_mode("regression")

knn_recipe <- recipe(played_hours ~ age + subscribe, data = clean_players)

lm_fit <- workflow() |>
    add_recipe(knn_recipe) |>
    add_model(knn_spec) |>
    fit(data = clean_players)
lm_fit

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)          age    subscribe  
    0.56610      0.02149      0.90817  


In [18]:
lm_test_results <- lm_fit |>
    predict(clean_players) |>
    bind_cols(clean_players) |>
    metrics(truth = played_hours, estimate = .pred) |>
    filter(.metric == "rmse") |>
    pull(.estimate)

lm_test_results

[1] 4.582584

## Subscribe Only Linear Regression

In [23]:
knn_spec <- linear_reg() |>
    set_engine("lm") |>
    set_mode("regression")

knn_recipe_2 <- recipe(played_hours ~ subscribe, data = clean_players)

lm_fit_2 <- workflow() |>
    add_recipe(knn_recipe_2) |>
    add_model(knn_spec) |>
    fit(data = clean_players)
lm_fit_2

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)    subscribe  
     1.0440       0.9357  


In [24]:
lm_test_results_subscribe <- lm_fit_2 |>
    predict(clean_players) |>
    bind_cols(clean_players) |>
    metrics(truth = played_hours, estimate = .pred) |>
    filter(.metric == "rmse") |>
    pull(.estimate)

lm_test_results_subscribe

[1] 4.59013

## Age Linear Regression Only

In [26]:
knn_spec <- linear_reg() |>
    set_engine("lm") |>
    set_mode("regression")

knn_recipe_3 <- recipe(played_hours ~ age, data = clean_players)

lm_fit_3 <- workflow() |>
    add_recipe(knn_recipe_3) |>
    add_model(knn_spec) |>
    fit(data = clean_players)
lm_fit_3

══ Workflow [trained] ══════════════════════════════════════════════════════════
Preprocessor: Recipe
Model: linear_reg()

── Preprocessor ────────────────────────────────────────────────────────────────
0 Recipe Steps

── Model ───────────────────────────────────────────────────────────────────────

Call:
stats::lm(formula = ..y ~ ., data = data)

Coefficients:
(Intercept)          age  
      1.198        0.023  


In [27]:
lm_test_results_age <- lm_fit_3 |>
    predict(clean_players) |>
    bind_cols(clean_players) |>
    metrics(truth = played_hours, estimate = .pred) |>
    filter(.metric == "rmse") |>
    pull(.estimate)

lm_test_results_age

[1] 4.600081

Cell Below Was used to create the quantile

In [ ]:
# Sample data
set.seed(123)
data <- data.frame(value = rnorm(100))

# Calculate the 97.5th percentile
cutoff <- quantile(data$value, 0.975)

# Filter out the top 2.5%
filtered_data <- data[data$value <= cutoff, ]

# Print the filtered data
print(filtered_data)
